In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from NoEmbedRNN import OpSel
from NoEmbedRNN import MemSel
from NoEmbedRNN import RNN as oldRNN
from NoEmbedRNN import MemRNN  as oldMemRNN
from NoEmbedRNN import HistoryRNN
from ops import Operations
from session import *
from data_gen import *
import pickle
from functools import reduce

modelName = 'RNN'
old = False
func = 'np_avg_val-5ops'
name = 'one_both_hardmaxes_have_sat_sat_grad_out/state_size_300-num_samples_3500-batch_size_100-learning_rate_0.001-epsilon_0.001-num_features_4-norm_True-softmax_sat_800-clip_False-share_state_True-rnns_same_state_False-state_fn_relu-pen_sofmax_True-smax_pen_r_0-seed_94169'
path = '/home/user/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
ops.ops = cfg["used_ops_obj"]
ops.num_of_ops = len(ops.ops)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        if old: mem = oldMemRNN(cfg, ops)
        else:   mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        if old: model = oldRNN(cfg, ops, mem)
        else:   model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'RNN_mem/Params/W_mem:0' shape=(304, 300) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(300,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W2_mem:0' shape=(300, 5) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(5,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W3_mem:0' shape=(5, 4) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b3_mem:0' shape=(4,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W:0' shape=(304, 300) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b:0' shape=(300,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W2:0' shape=(300, 5) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b2:0' shape=(5,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W3:0' shape=(5, 4) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b3:0' shape=(4,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'RNN_op/Grads/c

In [2]:
def print_errors(error_lst, name):
    print("")
    print("Total", reduce((lambda x, y: x + y), error_lst))
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [3]:
def print_matrix(total_error, matrix_lst, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
    print("")

In [4]:
def print_ops_matrix(total_error, matrix_lst, ops_list, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
    print("")

In [5]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")


Total 3532.42968523
sofmax_train_error[0] err is 2.11439374503e-28
sofmax_train_error[1] err is 1.24368852089e-28
sofmax_train_error[2] err is 2.24036497083e-28
sofmax_train_error[3] err is 3532.42968523
sofmax_train_error[4] err is 2.50863930836e-28
sofmax_train_error[5] err is 1.64477498739e-28
sofmax_train_error[6] err is 1.37261797508e-28
sofmax_train_error[7] err is 2.49674476502e-28
sofmax_train_error[8] err is 1.86960034537e-28
sofmax_train_error[9] err is 3.9494814258e-28
sofmax_train_error[10] err is 2.39813715187e-28
sofmax_train_error[11] err is 2.10231431241e-28
sofmax_train_error[12] err is 2.61901820533e-28
sofmax_train_error[13] err is 2.24825357988e-28
sofmax_train_error[14] err is 9.97909045105e-29
sofmax_train_error[15] err is 3.81808678127e-28
sofmax_train_error[16] err is 3.96057478228e-28
sofmax_train_error[17] err is 1.77099369519e-28
sofmax_train_error[18] err is 2.35080549756e-28
sofmax_train_error[19] err is 3.96008174421e-28
sofmax_train_error[20] err is 3.20

In [10]:
print_matrix(res["total_loss_traind_train"][0], res["softmaxes_traind_train"][0])
#print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"])

Error for this matrix is 2.11439374503e-28

##Elem-0--#############################################################
[ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  1.  0.  0.  0.] [ 0.  0.  1.  0.  0.] 
##Elem-1--#############################################################
[ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  1.  0.  0.  0.] [ 0.  0.  1.  0.  0.] 
##Elem-2--#############################################################
[ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  1.  0.  0.  0.] [ 0.  0.  1.  0.  0.] 
##Elem-3--#############################################################
[ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  1.  0.  0.  0.] [ 0.  0.  1.  0.  0.] 
##Elem-4--#############################################################
[ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  0.  1.  0.  0.] [ 0.  1.  0.  0.  0.] [ 0.  0.  1.  0.  0.] 
##Elem-5--###############################

In [7]:
#print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

Error for this matrix is 1.45150406561e-28

##Elem-0--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-1--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-2--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-3--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-4--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-5--#############################################################
[ tf_stall ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-6--###############################

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_train"][0])

In [ ]:
wrong_res = np.nonzero(np.round(np.apply_along_axis(np_add, 1, res["outputs_testd_train"][0][4] - res["batchesY_test"][0]), 2))[0]
wrong_res 

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_test_mem"][0])

In [ ]:
res["batchesX_test"][0]

In [ ]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res = predict_form_sess(model, cfg, x_test[0:100,], last_hardmax_state_train, model_path, mode="hard")

In [ ]:
res - y_test[0:100,]

In [ ]:
y_test[0:100,]

In [ ]:
res

In [ ]:
len(res["softmaxes_traind_train"])